# Explorando publicações do "Hacker News"

Neste projeto, compararemos dois tipos diferentes de postagens do **Hacker News**, um [site popular](https://news.ycombinator.com) em que histórias relacionadas à tecnologia são votadas e comentadas. Os dois tipos de postagens que exploraremos começam com *Ask HN* ou *Show HN*.

Os usuários enviam postagens do *Ask HN* para fazer à comunidade Hacker News uma pergunta específica, como "Qual é o melhor curso on-line que você já fez?". Da mesma forma, os usuários enviam postagens do *Show HN* para mostrar à comunidade do Hacker News um projeto, produto ou apenas algo geralmente interessante.

Vamos comparar especificamente esses dois tipos de postagens para determinar o seguinte:

1. *Ask HN* ou *Show HN* recebe mais comentários em média?
+ As postagens criadas em um determinado momento recebem mais comentários, em média?

É importante destacar que o conjunto de dados com o qual trabalhamos foi reduzido de quase 300.000 linhas para aproximadamente 20.000, removendo todos os envios que não receberam comentários e, em seguida, amostrando aleatoriamente os envios restantes.

## Introdução

Primeiro leremos os dados e separaremos cabeçalho de conteúdo. O primeiro será armazenado na variável `headers` enquanto que o segundo será armazenado na variável `hn`. Aproveitaremos para explorar o conteúdo de nosso conjunto de dados, exibindo o cabeçalho e as primeiras 4 linhas de conteúdo:

In [6]:
from csv import reader  # módulo para leitura de arquivos *.csv

# abre o arquivo e gera uma lista a partir de seu conteúdo
opened_file = open('hacker_news.csv')
hn = list(reader(opened_file))

# exibe cabeçalho e as primeiras 4 linhas de conteúdo
for row in hn[:5]:
    print(row)

# separa o cabeçalho do resto do conteúdo
headers = hn[0]
hn = hn[1:]

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


Podemos ver que o conjunto de dados contém, para cada postagem, o título, o número de comentários e a data de criação. Vamos começar explorando o número de comentários para cada tipo de postagem.

## Extraindo postagens dos tipos "Ask HN" e "Show HN"

Primeiro, identificaremos as postagens que começam com *Ask HN* ou *Show HN* e separaremos os dados desses dois tipos de postagens em listas diferentes. A separação dos dados facilita a análise nas etapas a seguir.

In [7]:
ask_posts = []
show_posts =[]
other_posts = []

for post in hn:
    title = post[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(post)
    elif title.lower().startswith("show hn"):
        show_posts.append(post)
    else:
        other_posts.append(post)
        
print(len(ask_posts), 'posts do tipo Ask HN.')
print(len(show_posts), 'posts do tipo Show HN.')
print(len(other_posts), 'posts de outras categorias.')

1744 posts do tipo Ask HN.
1162 posts do tipo Show HN.
17194 posts de outras categorias.


## Calculando o número médio de comentários

Agora que separamos as postagens solicitadas e as armazenamos em listas diferentes, calcularemos o número médio de comentários que cada tipo de postagem recebe.

In [8]:
# cálculo do numero médio de comentários nas postagens 'Ask HN'
total_ask_comments = 0

for post in ask_posts:
    total_ask_comments += int(post[4])
    
avg_ask_comments = total_ask_comments / len(ask_posts)

# cálculo do numero médio de comentários nas postagens 'Show HN'
total_show_comments = 0

for post in show_posts:
    total_show_comments += int(post[4])
    
avg_show_comments = total_show_comments / len(show_posts)

print('{:.2f} comentários, em média, nas postagens Ask HN.'.format(avg_show_comments))
print('{:.2f} comentários, em média, nas postagens Show HN '.format(avg_ask_comments))

10.32 comentários, em média, nas postagens Ask HN.
14.04 comentários, em média, nas postagens Show HN 


Em média, as postagens *Ask HN* em nossa amostra recebem aproximadamente 14 comentários, enquanto as postagens *Show HN* recebem aproximadamente 10 comentários. Como as postagens de pergunta têm mais probabilidade de receber comentários, focaremos nossa análise restante apenas nessas postagens.

## Localizando a quantidade de mensagens e comentários em posts, por hora de publicação

Em seguida, determinaremos se podemos maximizar a quantidade de comentários que uma postagem de solicitação recebe criando-a em um determinado momento. Primeiro, vamos encontrar a quantidade de postagens criadas durante cada hora do dia, juntamente com o número de comentários que essas postagens receberam. Em seguida, calcularemos a quantidade média de comentários que as postagens criadas a cada hora do dia recebem.

In [13]:
# importa módulo para tratamento de datas e horários
import datetime as dt

# armazena, em result_list, uma lista contendo,
# na ordem, os elementos ['created_at', 'num_comments'],
# para cada linha do dataset
result_list = []
for post in ask_posts:
    result_list.append(
        [post[6], int(post[4])]
    )

comments_by_hour = {}  # núm. de comentários a cada hora
counts_by_hour = {}  # núm de posts a cada hora
date_format = "%m/%d/%Y %H:%M"  # formato de data no qual os dados foram gravados

for row in result_list:
    date = row[0]
    comment = row[1]
    
    # recebe o dado de acordo com o template em 'date_format'
    # e retorna somente a hora do dia, armazenando-a em time
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    
    # cria as tabelas de frequências para comentários e 
    # para publicações a cada hora do dia
    if time not in counts_by_hour:
        comments_by_hour[time] = comment
        counts_by_hour[time] = 1        
    else:
        comments_by_hour[time] += comment
        counts_by_hour[time] += 1

print(comments_by_hour)
print(counts_by_hour)

{'00': 447, '01': 683, '11': 641, '14': 1416, '13': 1253, '05': 464, '23': 543, '08': 492, '07': 267, '09': 251, '21': 1745, '03': 421, '19': 1188, '15': 4477, '06': 397, '02': 1381, '10': 793, '22': 479, '20': 1722, '17': 1146, '12': 687, '16': 1814, '18': 1439, '04': 337}
{'00': 55, '01': 60, '11': 58, '14': 107, '13': 85, '05': 46, '23': 68, '08': 48, '07': 34, '09': 45, '21': 109, '03': 54, '19': 110, '15': 116, '06': 44, '02': 58, '10': 59, '22': 71, '20': 80, '17': 100, '12': 73, '16': 108, '18': 109, '04': 47}


Numa análise superficial, percebe-se que o maior número de comentáriose, em média, ocorre entre as 15:00 e as 16:00 (4.477 registros) e o maior número de publicações costuma se dar também neste horário.

## Número médio de comentários nas publicações 'Ask HN', por hora do dia

Seguiremos com nossa análise para determinar o número médio de comentários por post, para cada hora do dia.

In [15]:
avg_by_hour = []

for hour in comments_by_hour:
    average = comments_by_hour[hour] / counts_by_hour[hour]
    avg_by_hour.append([hour, average])

avg_by_hour

[['00', 8.127272727272727],
 ['01', 11.383333333333333],
 ['11', 11.051724137931034],
 ['14', 13.233644859813085],
 ['13', 14.741176470588234],
 ['05', 10.08695652173913],
 ['23', 7.985294117647059],
 ['08', 10.25],
 ['07', 7.852941176470588],
 ['09', 5.5777777777777775],
 ['21', 16.009174311926607],
 ['03', 7.796296296296297],
 ['19', 10.8],
 ['15', 38.5948275862069],
 ['06', 9.022727272727273],
 ['02', 23.810344827586206],
 ['10', 13.440677966101696],
 ['22', 6.746478873239437],
 ['20', 21.525],
 ['17', 11.46],
 ['12', 9.41095890410959],
 ['16', 16.796296296296298],
 ['18', 13.20183486238532],
 ['04', 7.170212765957447]]

Embora agora tenhamos os resultados que precisamos, esse formato dificulta a identificação das horas com os valores mais altos. Vamos terminar criando uma lista de listas e imprimindo os cinco valores mais altos em um formato mais fácil de ler.

In [17]:
swap_avg_by_hour = []

for row in avg_by_hour:
    # acrescentamos à lista swap_avg_by_hour os elementos
    # de avg_by_our na ordem [média, horário]
    swap_avg_by_hour.append([row[1], row[0]])
    
# agora podemos armazenar, em outra lista, os dados em ordem decrescente
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

sorted_swap

[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

De posse desses dados, podemos exibir os cinco horários com a maior média de comentários por publicação do tipo *Ask HN*:

In [18]:
for avg, hour in sorted_swap[:5]:
    print(
        '{}: {:.2f} comentários, em média, por post.'.format(
            dt.datetime.strptime(hour, "%H").strftime("%H:%M"), avg
        )
    )

15:00: 38.59 comentários, em média, por post.
02:00: 23.81 comentários, em média, por post.
20:00: 21.52 comentários, em média, por post.
16:00: 16.80 comentários, em média, por post.
21:00: 16.01 comentários, em média, por post.


Nota-se que os posts *Ask HN* recebem mais comentários entre as 15:00 e as 16:00, com uma média de 38,59 comentários a cada publicação. Se levarmos em consideração a proximidade entre os horários desses cinco primeiros registros, podemos dizer que, entre as 15:00 e as 16:00 e entre as 20:00 e 22:00, ocorrem os maiores picos de interação entre os leitores e os autores dos textos na comunidade.

## Conclusão

Neste projeto, analisamos as postagens de perguntas (*Ask HN*) e de demonstração (*Show HN*) do site **Hacker News** para determinar qual tipo de postagem e horário do dia recebem mais comentários em média. Com base em nossa análise, para maximizar a quantidade de comentários que uma postagem recebe, recomendamos que ela seja categorizada como *postagem de solicitação* e seja publicada entre 15:00 e 16:00.

No entanto, deve-se observar que o conjunto de dados que analisamos excluiu postagens sem nenhum comentário. Dado isso, é mais preciso dizer que, das postagens que receberam comentários, as postagens recebidas receberam mais comentários em média e as postagens criadas entre 15:00 e 16:00 receberam o maioria dos comentários em média.